
# IBM HR 데이터셋 — 페르소나 부여

IBM HR 데이터셋을 이용해 **직원 페르소나(Persona)** 를 자동으로 할당하는 과정을 단계별로 설명합니다.  

**무엇을 하는가?**
1. **데이터 로드**: CSV 파일(IBM_HR.csv)을 읽습니다.  
2. **간단 전처리**: 분석에 필요한 컬럼이 있는지 확인하고, `OverTime(Yes/No)`를 1/0으로 변환합니다.  
3. **정규화(0~1)**: 숫자 컬럼을 0~1 범위로 바꿔서 서로 다른 스케일을 맞춥니다.  
4. **페르소나 점수 계산**: 문서에 정의된 가중치와 방향(높을수록/낮을수록 위험)을 반영하여 **10개 페르소나(P01~P04, S01~S03, N01~N03)** 의 적합도 점수를 구합니다.  
5. **페르소나 할당**  
   - **결정적 할당(Argmax)**: 가장 점수가 높은 페르소나 1개를 고릅니다.  
   - **확률적 할당(Softmax 샘플링)**: 모든 페르소나 점수를 확률로 바꿔 **무작위로 1개**를 선택합니다(재현 가능하도록 시드 고정).  
6. **저장 및 미리보기**: 결과를 CSV로 저장하고, 일부 컬럼만 골라 미리 보여줍니다.

**입력 데이터 경로**: `./data/IBM_HR.csv`  
**출력 CSV 경로**: `./data/IBM_HR_personas_assigned.csv`


In [25]:

# ------------------------------------------------------
# 0) 라이브러리 임포트 및 경로/상수 설정
# ------------------------------------------------------
import pandas as pd
import numpy as np
from pathlib import Path

# 입력/출력 파일 경로
DATA_PATH = Path("./data/IBM_HR.csv")
OUTPUT_PATH = Path("./data/IBM_HR_personas_assigned.csv")

# 확률적 샘플링에 사용되는 소프트맥스 온도(낮을수록 가장 큰 점수에 더 쏠림)
SOFTMAX_TAU = 0.1  # 0.1~1.2 사이에서 조정 추천
RNG_SEED = 42      # 재현 가능성 확보(난수 시드 고정)


In [26]:

# ------------------------------------------------------
# 1) 데이터 로드
# ------------------------------------------------------
# - CSV 파일을 읽습니다. 파일이 없으면 에러를 내고 중단합니다.
# - 원본 보존을 위해 복사본(df_orig)을 따로 만듭니다.
if not DATA_PATH.exists():
    raise FileNotFoundError(f"데이터셋을 찾을 수 없습니다: {DATA_PATH}")

df = pd.read_csv(DATA_PATH)
df_orig = df.copy()

print("데이터 로드 완료:", df.shape)
df.head(3)  # 상위 3행 미리보기


데이터 로드 완료: (1470, 35)


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0


In [35]:

# ------------------------------------------------------
# 2) 간단 전처리 (필수 컬럼 확인, OverTime 변환, 정규화 준비)
# ------------------------------------------------------

# (1) 분석에 필요한 필수 컬럼이 모두 있는지 확인합니다.
required_cols = [
    # 이진/범주형
    "OverTime",
    # 숫자 컬럼(정규화 대상 및 점수 계산에 사용)
    "JobSatisfaction", "PerformanceRating", "JobInvolvement",
    "YearsAtCompany", "RelationshipSatisfaction",
    "YearsSinceLastPromotion", "TrainingTimesLastYear",
    "TotalWorkingYears", "NumCompaniesWorked",
    "PercentSalaryHike"
]

missing = [c for c in required_cols if c not in df.columns]
if missing:
    # 실무에서는 컬럼명 오타/스키마 차이가 흔합니다.
    # 이 에러를 보고 데이터 소스/컬럼명을 점검하세요.
    raise ValueError(f"필수 컬럼이 없습니다: {missing}")

# (2) OverTime: Yes/No -> 1/0 으로 변환
#    - 나중에 가중치를 곱해 점수를 만들 때 숫자형이 유리합니다.
if "OverTime_bin" not in df.columns:
    df["OverTime_bin"] = df["OverTime"].map({"Yes": 1, "No": 0})
    if df["OverTime_bin"].isna().any():
        df["OverTime_bin"] = (df["OverTime"] == "Yes").astype(int)

if "is_married" not in df.columns:
    df["is_married"] = (df["MaritalStatus"] == "Married").astype(int)

if "AgeFactor" not in df.columns:
    df["AgeFactor"] = ((df["Age"] >= 28) & (df["Age"] <= 45)).astype(int)

# (2) 이진/프록시 변수 생성
# OverTime: Yes/No → 1/0
df["OverTime_bin"] = df["OverTime"].map({"Yes": 1, "No": 0})
if df["OverTime_bin"].isna().any():
    df["OverTime_bin"] = (df["OverTime"] == "Yes").astype(int)

# MaritalStatus: Married → 1, 그 외(Single/Divorced 등) → 0
df["is_married"] = (df["MaritalStatus"] == "Married").astype(int)

# AgeFactor: 28~45세이면 1, 아니면 0 (신규 부모 페르소나의 대리 지표 보완)
df["AgeFactor"] = ((df["Age"] >= 28) & (df["Age"] <= 45)).astype(int)

# (3) 정규화 함수 정의 (Min-Max Scaling, 결과를 0~1 범위로)
#    - 서로 다른 단위/범위를 갖는 숫자들을 공정하게 비교하려면 스케일을 맞춰야 합니다.
#    - 모든 값이 같아 분모가 0이 되는 경우(상수열)는 0으로 반환하여 안정성 확보.
def min_max_norm(series: pd.Series) -> pd.Series:
    s = series.astype(float)
    s_min, s_max = s.min(), s.max()
    if pd.isna(s_min) or pd.isna(s_max) or s_max == s_min:
        return pd.Series(0.0, index=s.index)  # 상수열 또는 결측이면 0으로 반환
    return (s - s_min) / (s_max - s_min)

# (4) 정규화 적용할 컬럼들 매핑(원본명 -> 정규화 컬럼명)
norm_map = {
    "JobSatisfaction": "Norm_JobSatisfaction",
    "WorkLifeBalance": "Norm_WorkLifeBalance",
    "PerformanceRating": "Norm_PerformanceRating",
    "YearsAtCompany": "Norm_YearsAtCompany",
    "RelationshipSatisfaction": "Norm_RelationshipSatisfaction",
    "EnvironmentSatisfaction": "Norm_EnvironmentSatisfaction",
    "YearsSinceLastPromotion": "Norm_YearsSinceLastPromotion",
    "TrainingTimesLastYear": "Norm_TrainingTimesLastYear",
    "JobInvolvement": "Norm_JobInvolvement",
    "MonthlyIncome": "Norm_MonthlyIncome",
    "PercentSalaryHike": "Norm_PercentSalaryHike",
    "JobLevel": "Norm_JobLevel",
    "TotalWorkingYears": "Norm_TotalWorkingYears",
    "NumCompaniesWorked": "Norm_NumCompaniesWorked",
}

# (5) 누락된 정규화 컬럼만 생성
missing_norms = []
for src, dst in norm_map.items():
    if dst not in df.columns:
        if src not in df.columns:
            missing_norms.append((src, dst))  # 원본도 없으면 보고
        else:
            df[dst] = min_max_norm(df[src])

# 5) 점검 메시지
expected_norms = list(norm_map.values())
still_missing = [c for c in expected_norms if c not in df.columns]
if still_missing:
    print("⚠️ 여전히 누락된 정규화 컬럼:", still_missing)
    print("→ 데이터셋의 원본 컬럼명이 다른지 확인하세요. (예: 철자/대소문자/공백)")
else:
    print("✅ 정규화 컬럼 준비 완료:", len(expected_norms), "개")


✅ 정규화 컬럼 준비 완료: 14 개



### 페르소나 적합도 점수 산정 방법론

#### 페르소나 유형 정의

#### 유형 구분1 : 고위험 군
- 이 유형은 이직 가능성이 높은 직원들의 주요 시나리오를 대표하며, 각 페르소나는 앞서 살펴본 조직 심리학 이론에서 검증된 특정 이직 동인과 직접적으로 연결됨.

**P01: 번아웃에 직면한 직원 (High Demands, Low Resources)**
- 이 페르소나는 JD-R 모델의 '건강 손상 과정(health impairment process)'을 전형적으로 보여줌.
- 이들은 만성적으로 높은 직무 요구(예: 과도한 초과 근무, 높은 업무 압박)에 직면해 있지만, 이를 완충할 충분한 직무 자원(예: 자율성, 상사의 지지, 인정)이 부족한 상태에 놓여 있음.
- 지속적인 에너지 고갈은 결국 정서적 소진과 냉소주의로 이어져 이직의 주요 원인이 됨.

**P02: 온보딩에 실패한 직원 (Early-Stage Disengagement)**
- 이 유형은 조직의 초기 단계에서 발생하는 이직 위험을 대표함.
- 입사 전 기대와 조직의 현실 간의 괴리가 주된 원인임.
- 역할의 불명확성, 조직 문화 부적응, 또는 초기 지원 시스템의 부재로 인해 자기결정성 이론에서 강조하는 유능성과 관계성의 기본적 욕구가 좌절되며, 이는 조기 이탈로 이어질 가능성 高.

**P03: 성장이 정체된 직원 (Career Plateauing)**

- 성장 및 발전 기회의 부재로 인해 경력이 정체된 상태에 놓인 직원.
- 이 페르소나의 동기 저하는 허즈버그의 2요인 이론(Herzberg's Two-Factor Theory)에서 말하는 '동기 요인(motivators)'인 승진, 성장, 인정의 결핍에서 비롯됨.
- 반복적인 업무와 비전 부재는 자기결정성 이론의 '유능성' 욕구를 충족시키지 못하며, 이는 내재적 동기의 점진적인 침식과 이직 탐색 행동으로 이어짐.

**P04: 저평가된 전문가 (Compensation-Driven Risk)**
- 이 직원의 핵심적인 불만 요인은 보상의 불공정성에 대한 인식.
- 뛰어난 기술과 생산성을 보유하고 있을 수 있으나, 이들의 동기는 주로 외재적 보상에 의해 좌우됨.
- 외부 노동 시장의 연봉 수준과 자신의 보상을 비교하며 상대적 박탈감을 느끼기 쉬우며, 더 나은 금전적 제안이 있을 경우 쉽게 이직을 결심하는 '유인 요인(pull factor)'에 취약한 유형.

#### 유형 구분2 : 안정 및 몰입 군
- 이 유형은 조직의 안정성과 생산성에 긍정적으로 기여하는 직원들의 특성을 나타냄.
- 이들은 높은 수준의 직무 자원을 보유하고 있으며, 이는 높은 수준의 업무 몰입으로 이어짐.

**S01: 안정적인 핵심인재 (Loyal High-Performer)**
- 이 페르소나는 조직의 안정성을 지탱하는 핵심 인력.
- 오랜 근속 연수, 높은 신뢰도, 꾸준한 성과가 특징.
- 이들의 동기는 조직의 미션과 가치에 깊이 내재화되어 있으며, JD-R 모델의 '동기 부여 과정(motivational process)'이 성공적으로 작동하는 대표적인 사례로, 풍부한 자원이 높은 몰입과 조직 헌신으로 이어진 결과.

**S02: 라이징 스타 (High-Potential & Motivated)**
- 가파른 성장 곡선을 그리고 있는 高성과, 高잠재력 인재.
- 높은 성과 평가, 잦은 승진, 그리고 높은 직무 몰입도가 이들의 주요 지표.
- 이들은 자기결정성 이론의 유능성과 성취에 대한 욕구가 강하게 충족되고 있으며, 내재적 동기(성취감, 인정)와 외재적 동기(승진, 보상)가 모두 긍정적으로 작용하여 높은 수준의 몰입을 보임.

**S03: 전문가 (Intrinsically Motivated Expert)**
- 업무 그 자체에서 깊은 만족감을 얻는 유형.
- 이들의 동기는 주로 내재적이며, 구체적으로는 자신의 전문 분야에 대한 열정과 일의 의미에 의해 동기가 부여됨.
- 이들은 합리적인 수준의 보상이 주어진다면 외재적 요인(예: 급여 인상)에 덜 민감하며, 자율성과 전문성 발휘의 기회를 더 중요하게 여김.

#### 유형 구분3 : 중립 및 관망 군
- 이 유형은 명확한 이직 위험이나 높은 몰입 상태를 보이지 않는, 중간 영역에 위치한 직원들을 포함.
- 이들의 상태는 잠재적으로 긍정적 또는 부정적 방향으로 전환될 수 있어 세심한 관리가 필요.

**N01: 현상만 유지하는 직원 (Maintains Status Quo)**
- 주어진 직무 기대치는 충족시키지만, 그 이상의 주도성이나 추가적인 기여에 대한 동기는 부족.
- 이들의 동기는 허즈버그의 2요인 이론(Herzberg's Two-Factor Theory)에서 말하는 ’위생 요인(hygiene factors)'(ex. 급여)에 의해 유지되며, 적극적인 몰입 없이 중립적인 만족 상태에 머물러 있음.

**N02: 유능하지만 불만이 많은 직원 (The Competent but Disengaged)**
- 높은 업무 역량을 바탕으로 성과를 창출하지만, 조직에 대한 정서적 연결이나 관계적 만족도는 낮은 상태.
- 높은 성과 평가에도 불구하고 낮은 관계 만족도나 직무 몰입도를 보일 수 있음.
- 이들의 이직 위험은 역량 부족이 아닌, 문화적 부적합이나 더 나은 개인적 기회가 주어졌을 때의 갑작스러운 이탈 가능성에 있음.

**N03: 신규 부모 (The New Parent)**
- 이 페르소나는 출산이나 새로운 부양 책임과 같은 중요한 개인적인 이벤트로 인해 워라벨에 대한 우선순위가 일시적으로 변화하는 단계를 겪고 있는 직원.
- 본질적으로 고위험군이나 저몰입 직원은 아니지만, 유연 근무나 지원 시스템과 같은 직무 자원에 대한 필요성이 급격히 증가함.
- 이 페르소나는 삶의 주요 사건이 직무 행동에 미치는 영향을 모델이 학습할 수 있도록 하는 중요한 역할을 함.


In [36]:

# ------------------------------------------------------
# 3) 페르소나 적합도 점수 계산 (가중합)
# ------------------------------------------------------

# (참고) 각 항은 0~1 범위입니다.
# - 일반적으로 지표가 "높을수록 해당 페르소나에 더 가깝다"면 Norm_x 사용
# - "낮을수록 더 가깝다"면 (1 - Norm_x) 사용
# - 이진은 OverTime_bin 처럼 0/1 값 그대로 사용

# 가독성을 위해 각 페르소나를 별도 변수로 계산한 뒤 DataFrame으로 합칩니다.

# 고위험군 (P01~P04)
P01 = (
    0.4 * df["OverTime_bin"] +
    0.3 * (1 - df["Norm_JobSatisfaction"]) +
    0.2 * (1 - df["Norm_WorkLifeBalance"]) +
    0.1 * (1 - df["Norm_PerformanceRating"])
)

P02 = (
    0.5 * (1 - df["Norm_YearsAtCompany"]) +
    0.3 * (1 - df["Norm_RelationshipSatisfaction"]) +
    0.2 * (1 - df["Norm_EnvironmentSatisfaction"])
)

P03 = (
    0.5 * df["Norm_YearsSinceLastPromotion"] +
    0.3 * (1 - df["Norm_TrainingTimesLastYear"]) +
    0.2 * (1 - df["Norm_JobInvolvement"])
)

P04 = (
    0.4 * (1 - df["Norm_MonthlyIncome"]) +
    0.4 * (1 - df["Norm_PercentSalaryHike"]) +
    0.2 * df["Norm_JobLevel"]
)

# 안정·몰입군 (S01~S03)
S01 = (
    0.4 * df["Norm_YearsAtCompany"] +
    0.4 * df["Norm_TotalWorkingYears"] +
    0.2 * (1 - df["Norm_NumCompaniesWorked"])
)

S02 = (
    0.4 * df["Norm_PerformanceRating"] +
    0.4 * (1 - df["Norm_YearsSinceLastPromotion"]) +
    0.2 * df["Norm_PercentSalaryHike"]
)

S03 = (
    0.4 * df["Norm_JobInvolvement"] +
    0.4 * df["Norm_JobSatisfaction"] +
    0.2 * df["Norm_JobLevel"]
)

# 중립·관망군 (N01~N03)
N01 = (
    0.5 * (1 - df["Norm_JobInvolvement"]) +
    0.3 * (1 - df["Norm_PerformanceRating"]) +
    0.2 * (1 - df["Norm_PercentSalaryHike"])
)

N02 = (
    0.5 * df["Norm_PerformanceRating"] +
    0.3 * (1 - df["Norm_RelationshipSatisfaction"]) +
    0.2 * (1 - df["Norm_EnvironmentSatisfaction"])
)

N03 = (
    0.5 * df["is_married"] +
    0.3 * df["AgeFactor"] +
    0.2 * (1 - df["OverTime_bin"])
)


# 점수 테이블로 결합
scores_df = pd.DataFrame({
    "P01": P01, "P02": P02, "P03": P03, "P04": P04,
    "S01": S01, "S02": S02, "S03": S03,
    "N01": N01, "N02": N02, "N03": N03
}, index=df.index)

scores_df.head(3)



,P01,P02,P03,P04,S01,S02,S03,N01,N02,N03
0,0.700000,0.858333,0.366667,0.745018,0.162222,0.400000,0.716667,0.666667,0.433333,0.3
1,0.266667,0.441667,0.316667,0.420339,0.377778,0.944762,0.316667,0.361905,0.566667,0.7
2,0.666667,0.700000,0.283333,0.662944,0.136667,0.457143,0.400000,0.776190,0.200000,0.3



## 4) 페르소나 할당: 결정적 + 확률적

- **결정적(Argmax)**: 가장 점수가 높은 페르소나 1개를 그대로 선택  
- **확률적(Softmax 샘플링)**: 모든 점수를 확률로 바꾸어 무작위로 1개를 선택  
  - 온도(**τ**)를 낮추면 가장 큰 점수에 더 쏠리고, 높이면 다양성이 커집니다.


In [37]:

# ------------------------------------------------------
# 4) 페르소나 점수 계산 및 할당
# ------------------------------------------------------

# (1) 결정적 할당: argmax
argmax_code = scores_df.idxmax(axis=1)

# (2) 확률적 할당을 위한 softmax 함수
def softmax(arr, tau=1.0, axis=1):
    # 수치 안정성을 위해 최대값을 빼고 exp를 취합니다.
    # 지수( exp )는 입력이 크면 overflow가 나기 쉽습니다.
    # 각 행에서 최대값을 빼고 지수를 취하면, 가장 큰 값은 exp(0)=1, 나머지는 exp(음수)<1이 되어 안정적입니다.
    m = np.max(arr, axis=axis, keepdims=True)
    ex = np.exp((arr - m) / tau)
    return ex / np.sum(ex, axis=axis, keepdims=True)

probs = softmax(scores_df.values, tau=SOFTMAX_TAU, axis=1)
probs_df = pd.DataFrame(
    probs, columns=[f"Prob_{c}" for c in scores_df.columns], index=df.index
)

# (3) 확률적 샘플링: 확률분포(probs)에 따라 1개 페르소나 코드를 선택
rng = np.random.default_rng(RNG_SEED)  # 재현 가능한 난수 생성기(default_rng()는 NumPy의 난수 생성기인 Generator를 반환)
codes = scores_df.columns.to_list()

def softmax_code(prob_row):
    return rng.choice(codes, p=prob_row)  # p=prob_row는 합이 1이어야 함(softmax가 보장)

softmax_codes = np.apply_along_axis(softmax_code, 1, probs)

# 라벨(이름/리스크 티어) 매핑
persona_name = {
    "P01": "번아웃에 직면한 직원",
    "P02": "온보딩에 실패한 직원",
    "P03": "성장이 정체된 직원",
    "P04": "저평가된 전문가",
    "S01": "안정적인 핵심인재",
    "S02": "라이징 스타",
    "S03": "내재 동기 전문가",
    "N01": "현상 유지자",
    "N02": "유능하지만 불만이 많은 직원",
    "N03": "신규 부모",
}
persona_risk_tier = {
    "P01": "High", "P02": "High", "P03": "High", "P04": "High",
    "S01": "Stable", "S02": "Stable", "S03": "Stable",
    "N01": "Neutral", "N02": "Neutral", "N03": "Neutral",
}

argmax_name = argmax_code.map(persona_name)
argmax_tier = argmax_code.map(persona_risk_tier)

softmax_names = pd.Series(softmax_codes, index=df.index).map(persona_name)
softmax_tiers = pd.Series(softmax_codes, index=df.index).map(persona_risk_tier)

# 분포 확인(각 페르소나가 차지하는 비율)
print("결정적 할당 분포(비율):")
display(pd.Series(argmax_code).value_counts(normalize=True).rename("Assigned share").sort_index().to_frame())

print("확률적 샘플링 분포(비율):")
display(pd.Series(softmax_codes).value_counts(normalize=True).rename("Sampled share").sort_index().to_frame())


결정적 할당 분포(비율):


,Assigned share
N01,0.172789
N02,0.019728
N03,0.273469
P01,0.068707
P02,0.125850
P03,0.005442
P04,0.130612
S01,0.012245
S02,0.093197
S03,0.097959


확률적 샘플링 분포(비율):


,Sampled share
N01,0.174150
N02,0.026531
N03,0.231973
P01,0.074830
P02,0.151701
P03,0.012245
P04,0.133333
S01,0.018367
S02,0.089796
S03,0.087075



## 5) 결과 저장 및 미리보기

- `scores`(페르소나 점수), `probs`(확률), `argmax_*`(결정적), `softmax_*`(확률적)를 **원본 데이터에 합쳐서** CSV로 저장합니다.


In [ ]:

# ------------------------------------------------------
# 5) 저장 및 미리보기
# ------------------------------------------------------

# 최종 결과 테이블 구성
result = df.copy()

# 점수/확률 붙이기
result = pd.concat([
    result,
    scores_df.add_prefix("Score_"),
    probs_df
], axis=1)

# 결정적 & 확률적 결과 붙이기
result["argmax_Persona_Code"] = argmax_code
result["argmax_Persona"] = argmax_name
result["argmax_RiskTier"] = argmax_tier

result["softmax_Persona_Code"] = softmax_codes
result["softmax_Persona"] = softmax_names
result["softmax_RiskTier"] = softmax_tiers

# CSV로 저장
result.to_csv(OUTPUT_PATH, index=False)
print("저장 완료:", OUTPUT_PATH)

# 미리보기(핵심 컬럼 위주)
preview_cols = [
    # 원본 핵심
    "OverTime", "JobSatisfaction", "WorkLifeBalance", "PerformanceRating",
    "YearsAtCompany", "RelationshipSatisfaction", "EnvironmentSatisfaction",
    "YearsSinceLastPromotion", "TrainingTimesLastYear", "JobInvolvement",
    "MonthlyIncome", "PercentSalaryHike", "JobLevel", "TotalWorkingYears",
    "NumCompaniesWorked", "MaritalStatus", "Age",
    # 파생
    "OverTime_bin", "is_married", "AgeFactor",
    # 점수
    "Score_P01","Score_P02","Score_P03","Score_P04",
    "Score_S01","Score_S02","Score_S03",
    "Score_N01","Score_N02","Score_N03",
    # 할당
    "argmax_Persona_Code","argmax_Persona","argmax_RiskTier",
    "softmax_Persona_Code","softmax_Persona","softmax_RiskTier"
]
result[preview_cols].head(12)


저장 완료: data\IBM_HR_personas_assigned.csv


KeyError: "['OverTime_bin', 'is_married', 'AgeFactor'] not in index"


---

### 추가 팁
- **온도(τ)**: `SOFTMAX_TAU`를 0.1~1.2로 조정하며 확률 분포의 다양성을 조절할 수 있습니다.  
- **가중치 변경**: 특정 조직/도메인에 더 맞게 만들려면 위의 가중치를 조정하세요.  
- **품질 점검**: 
  - `argmax_Persona`의 분포가 특정 집단에 과도하게 쏠리지 않는지 확인  
  - (있다면) 실제 Attrition과의 상관을 보며 가중치를 보정  